In [ ]:
import os

os.chdir("..")

In [ ]:
import geopandas as gpd
import polars as pl

In [ ]:
gdf = gpd.read_parquet("data/external/geo-us-states.parquet")

In [ ]:
df = pl.DataFrame(gdf[["REGION","DIVISION","STATEFP","STUSPS","NAME"]])

In [ ]:
df.rename({
    "REGION":"region_id",
    "DIVISION":"division_id",
    "STATEFP":"state_fips",
    "STUSPS":"name_abv",
    "NAME":"name_full"
})

In [ ]:
df.write_database(table_name="states",if_table_exists="append",connection="sqlite:///database.db")